In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
from datetime import datetime

# === 1. Đọc dữ liệu ===
df_grade = pd.read_csv("../data/udk_moodle_all_grades.csv")
df_log = pd.read_csv("../data/udk_moodle_log_filtered.csv")

# === 2. Chọn khóa học ===
course_id = 670

# --- Dữ liệu điểm ---
df_course_grade = df_grade[(df_grade['courseid'] == course_id) & (df_grade['itemtype'] == 'course')].copy()

# --- Dữ liệu sự kiện ---
df_course_log = df_log[df_log['courseid'] == course_id].copy()

# === 3. Thống kê cơ bản ===
num_students = df_course_grade['userid'].nunique()
num_events = len(df_course_log)
num_event_types = df_course_log['eventname'].nunique()

# === 4. Thống kê điểm ===
grade_stats = df_course_grade['finalgrade'].describe().to_dict()

# === 5. Thống kê top sự kiện và hành động ===
top_events = df_course_log['eventname'].value_counts().head(10).to_dict()
top_actions = df_course_log['action'].value_counts().head(10).to_dict()

# === 6. Ghép log & điểm để phân tích tương quan ===
df_merge = pd.merge(df_course_log, df_course_grade[['userid', 'finalgrade']], on='userid', how='inner')
activity_per_user = df_merge.groupby('userid').size().reset_index(name='event_count')
activity_grade = pd.merge(activity_per_user, df_course_grade[['userid', 'finalgrade']], on='userid', how='inner')
corr_value = activity_grade['event_count'].corr(activity_grade['finalgrade'])

# === 7. Tạo cấu trúc dữ liệu tổng hợp ===
course_summary = {
    "course_id": int(course_id),
    "summary": {
        "num_students": int(num_students),
        "num_events": int(num_events),
        "num_event_types": int(num_event_types),
        "analysis_date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    },
    "grades": {
        "statistics": grade_stats,
        "top_5_students_highest": df_course_grade.nlargest(5, "finalgrade")[["userid", "finalgrade"]].to_dict(orient="records"),
        "top_5_students_lowest": df_course_grade.nsmallest(5, "finalgrade")[["userid", "finalgrade"]].to_dict(orient="records"),
    },
    "logs": {
        "top_10_events": top_events,
        "top_10_actions": top_actions,
        "example_log_records": df_course_log.head(5).to_dict(orient="records"),
    },
    "activity_analysis": {
        "students_activity_summary": activity_grade.describe().to_dict(),
        "correlation_event_vs_grade": float(corr_value)
    },
    "insights": {
        "high_activity_high_grade": bool(corr_value > 0.3),
        "low_activity_low_grade": bool(corr_value < 0),
        "notes": "Hệ số tương quan dương cho thấy sinh viên hoạt động nhiều có xu hướng đạt điểm cao hơn."
                  if corr_value > 0 else
                  "Hệ số tương quan âm cho thấy sinh viên hoạt động nhiều nhưng điểm thấp, cần xem lại chất lượng hoạt động."
    }
}

# === 8. Xuất file JSON ===
output_json_path = f"../data/course_{course_id}_analysis.json"
with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(course_summary, f, indent=4, ensure_ascii=False)

print(f"✅ Đã lưu kết quả phân tích chi tiết vào file: {output_json_path}")
print(json.dumps(course_summary, indent=2, ensure_ascii=False)[:1200], "\n... (đã cắt ngắn hiển thị)")

✅ Đã lưu kết quả phân tích chi tiết vào file: ../data/course_670_analysis.json
{
  "course_id": 670,
  "summary": {
    "num_students": 21,
    "num_events": 13995,
    "num_event_types": 65,
    "analysis_date": "2025-10-08 06:44:10"
  },
  "grades": {
    "statistics": {
      "count": 21.0,
      "mean": 3.4761904761904763,
      "std": 4.41156488809864,
      "min": -1.0,
      "25%": -1.0,
      "50%": 6.0,
      "75%": 8.0,
      "max": 9.0
    },
    "top_5_students_highest": [
      {
        "userid": 8609,
        "finalgrade": 9
      },
      {
        "userid": 9169,
        "finalgrade": 8
      },
      {
        "userid": 9152,
        "finalgrade": 8
      },
      {
        "userid": 9109,
        "finalgrade": 8
      },
      {
        "userid": 9151,
        "finalgrade": 8
      }
    ],
    "top_5_students_lowest": [
      {
        "userid": 9227,
        "finalgrade": -1
      },
      {
        "userid": 7267,
        "finalgrade": -1
      },
      {
        